In [ ]:
import cv2
cv2.__version__

'4.1.2'

In [1]:
import cv2
import numpy as np
import os
import os.path
from google.colab.patches import cv2_imshow

from skimage.feature import hog

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
# Menetukan diretory gambar
lokasi_gambar_pos = "drive/MyDrive/Dataset Baru/"
# tampungan = []
# x = []

# Menentukan ukuran tinggi dan lebar dari gambar
tinggi = 200
lebar = 200
dim = (lebar, tinggi)

# buat variable penampung array citra
pos_images = []
# neg_images = []

# Mengumpulkan data citra yang akan dibuka dalam satu array
for path, sub_directory, files in os.walk(lokasi_gambar_pos):
    for name_file in files:
      # Gabungkan nama file dengan lokasi gambar
      img_path = (os.path.join(path,name_file))
      if img_path.endswith('png'):
        # Membaca gambar dari img_path
        gambar = cv2.imread(img_path)
        # Preprocessing data / segmentasi
        convert = cv2.cvtColor(gambar,cv2.COLOR_BGR2GRAY)
        rezise = cv2.resize(convert,dim)
        blur = cv2.GaussianBlur(rezise,(5,5),0)   
        pos_images.append(blur.flatten())

In [10]:
# Buat variable labeling
pos_labels = np.zeros(114)
neg_labels = np.ones(70)
api_labels = np.full((559),2)

x = np.array(pos_images)
y = np.concatenate((pos_labels,neg_labels,api_labels),axis=0)

print(x.shape)
print(y)

print("Shape of image set",x.shape)
print("Shape of labels",y.shape)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42, stratify=y)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(743, 40000)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2

Membangun Model dengan SVM

In [11]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix

In [12]:
# membangun model SVM
model = svm.SVC(C=1.0, kernel="linear" ,decision_function_shape='ovr', probability=True)
score = cross_validate(model, x, y, scoring='accuracy',
                         cv=5, return_train_score=True)
print(score['test_score'].mean())

0.9461636132777071


Melakukan Uji Coba data

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Melakukan uji coba
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print("Accuracy score of model is ",accuracy_score(y_pred=y_pred,y_true=y_test)*100)
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Accuracy score of model is  98.65771812080537
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98        23
         1.0       1.00      0.93      0.96        14
         2.0       0.99      0.99      0.99       112

    accuracy                           0.99       149
   macro avg       0.98      0.97      0.98       149
weighted avg       0.99      0.99      0.99       149



array([[ 23,   0,   0],
       [  0,  13,   1],
       [  1,   0, 111]])

Membangun model dengan algoritma KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

model = KNeighborsClassifier(n_neighbors=1)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

akurasi = accuracy_score(y_test, y_pred)*100
presisi = precision_score(y_test, y_pred, average='macro')*100
recall =recall_score(y_test, y_pred, average='macro')*100

print('akurasi: '+ str(akurasi))
print('presisi: '  +str(presisi))
print('recall: '  +str(recall))

akurasi: 100.0
presisi: 100.0
recall: 100.0


In [ ]:
from ctypes import resize
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(x,y)

# Menetukan diretory gambar
lokasi_gambar_pos = "drive/MyDrive/Dataset Telur Asin/telur/opencv_frame_114.png"

#Hasil Uji Klasifikasi pada dataset
Categories = ['Api','Bukan','Telur Asin']


# Membaca gambar dari img_path
gambar = cv2.imread(img_path)
# Preprocessing data / segmentasi
convert = cv2.cvtColor(gambar,cv2.COLOR_BGR2GRAY)
rezise = cv2.resize(convert,dim)
blur = cv2.GaussianBlur(rezise,(5,5),0) 
l = [blur.flatten()]
print(l)
# Mengecek hasil klasifikasi pada salah satu dataset
probability = model.predict_proba(l)
for ind,val in enumerate(Categories):
    print(f'{val} = {probability[0][ind]*100}%')
hasil = model.predict(l).astype(int)
print("The predicted image is : "+str(hasil)+Categories[int(hasil)])

[array([90, 90, 91, ..., 96, 94, 94], dtype=uint8)]
Bukan = 0.0%
Telur Asin = 100.0%
The predicted image is : [1]Telur Asin


Melukukan uji coba dengan gambar

In [15]:
# Menetukan diretory gambar
lokasi_gambar_pos = "drive/MyDrive/Testing"

#Hasil Uji Klasifikasi pada dataset
Categories = ['Bukan','Telur Asin','Api']

model.fit(x,y)

# Mengumpulkan data citra yang akan dibuka dalam satu array
for path, sub_directory, files in os.walk(lokasi_gambar_pos):
    for name_file in files:
      # Gabungkan nama file dengan lokasi gambar
      img_path = (os.path.join(path,name_file))
      if img_path.endswith('png'):
        # Membaca gambar dari img_path
        gambar = cv2.imread(img_path)
        # Preprocessing data / segmentasi
        convert = cv2.cvtColor(gambar,cv2.COLOR_BGR2GRAY)
        rezise = cv2.resize(convert,dim)
        blur = cv2.GaussianBlur(rezise,(5,5),0)   
        l = [blur.flatten()]
        probability = model.predict_proba(l)
        id_= int(model.predict(l)[0]) #prediksi
        name = Categories[id_]+' -> '+ str(probability[0][id_]) #tulisan buat hasil
        if probability.all() >= 0.5:
            print(name)
            cv2_imshow(gambar)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Menetukan diretory gambar
lokasi_vidio = "drive/MyDrive/Sistem Pendeteksi Gelas.mp4"

# Membuat variable untuk conter img
img_counter = 0

# Menentukan ukuran tinggi dan lebar dari gambar
tinggi = 200
lebar = 300
dim = (lebar, tinggi)

# Buat objek VideoCapture dan baca dari file input
cap = cv2.VideoCapture(lokasi_vidio)

# Periksa apakah kamera berhasil dibuka
if (cap.isOpened()== False): 
  print("Error opening video  file")

Error opening video  file
